<a href="https://colab.research.google.com/github/Datasci266-Final-Project/Job-Listings/blob/main/Model_2_Tokenizer_xlm_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Installs

!pip install pydot --quiet
!pip install transformers==4.37.2 --quiet
!pip install -U imbalanced-learn --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 5.5 MB/s eta 0:00:00


In [ ]:
#@title Imports
import pandas as pd

import numpy as np
import random
import torch

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, AutoTokenizer

from transformers import logging
logging.set_verbosity_error()

from collections import Counter

import pickle


In [ ]:
# Set seed to 10

seed_value = 10

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/DataSci 266 Project



Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1V3QooLePiHR_DaZhbXQhsjmP1Ez5fv5F/DataSci 266 Project


In [ ]:
pd.set_option('display.max_colwidth', 10000)

In [ ]:
#@title import pre-split datasets
with open(r'Model Objects/model_2_train_final_df.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open(r'Model Objects/model_2_test_data.pickle', 'rb') as handle:
    test_data = pickle.load(handle)
with open(r'Model Objects/model_2_val_data.pickle', 'rb') as handle:
    val_data = pickle.load(handle)

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(59222, 8)
(14063, 8)
(14063, 8)


In [ ]:
#@title BERT

checkpoint = 'FacebookAI/xlm-roberta-base'
bert_tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
print("Final class distributions for train dataset:")
print(train_data['level_labels'].value_counts())
print(train_data['salary_labels'].value_counts())
len(train_data)

Final class distributions for train dataset:
level_labels
 1    9310
 0    9310
 5    9284
 3    9152
-1    8627
 4    6786
 2    6753
Name: count, dtype: int64
salary_labels
-1    48941
 0     3255
 1     3255
 2     1257
 3     1257
 4     1257
Name: count, dtype: int64


59222

In [ ]:
#@title Tokenize
#ONLY need to tokenize train data, since that was re under/over sampled

texts_train = train_data['text'].tolist()

MAX_SEQUENCE_LENGTH = 512

bert_train_tokenized = bert_tokenizer(
    texts_train,
    max_length=MAX_SEQUENCE_LENGTH,
    truncation=True,
    padding='max_length',
    return_tensors='tf'
)

bert_train_inputs = [bert_train_tokenized.input_ids,
                     bert_train_tokenized.attention_mask]


In [ ]:
train_data['level_labels'] = train_data['level_labels'].astype(int)
train_data['salary_labels'] = train_data['salary_labels'].astype(int)

test_data['level_labels'] = test_data['level_labels'].astype(int)
test_data['salary_labels'] = test_data['salary_labels'].astype(int)

val_data['level_labels'] = val_data['level_labels'].astype(int)
val_data['salary_labels'] = val_data['salary_labels'].astype(int)

In [ ]:
#@title Export pickled tokenized inputs

#export tokenized inputs
with open('model_2_train_inputs_xlm_roberta.pickle', 'wb') as handle:
    pickle.dump(bert_train_inputs, handle, protocol=pickle.HIGHEST_PROTOCOL)





